In [ ]:
# load the packages needed
import numpy as np
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from src.solver import Solver
from src.model import AR, MA
import src.data_processor as dp
import src.inference as inf
from src.ts_gen import ar1_gen
data = np.loadtxt("../data/GOOG.csv", delimiter=',')

In [ ]:
X = np.array([data[0:100]])
Y = dp.get_return(X)

In [ ]:
lag = 2
sigma = 1.0
intercept = 0.1
phi = np.random.randn(2, 1)
AR_model = AR(lag=lag, phi=phi, sigma=sigma, intercept=intercept)
AR_model.params

In [ ]:
_, grads = AR_model.loss(Y)
solver = Solver(AR_model, Y,
                          update_rule='sgd_momentum',
                          optim_config={
                            'learning_rate': 1e-6,
                          },
                          
                          num_epochs=1000, batch_size=1,
                          print_every=10)
solver.train()

In [ ]:
AR_model.params

In [ ]:
a, b = inf.yule_walker(Y, order =2,method='unbiased')
print a
print b

In [ ]:
rt = AR_model.predict(Y,10)
print rt
pred_price = dp.get_price(X[0,99],rt)
print pred_price
print data[100:110]

In [ ]:

import matplotlib.pyplot as plt

# This is a bit of magic to make matplotlib figures appear inline in the notebook
# rather than in a new window.
%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

plt.plot(pred_price[0,:],'b')
plt.plot(data[100:110],'r')
plt.hold(True)
plt.show()